# Subqueries Introduction
- Subqueries allow us to break more complex queries into parts
	- Best suited for straightforward two-step queries
- Consists of a nested inner query (temp table) referenced by outer query
	- The inner query can be referenced like a table or view

In [1]:
USE AW2019;

SELECT TOP 10
    PurchaseOrderID,
    VendorID,
    OrderDate,
    TaxAmt,
    Freight,
    TotalDue,
    TotalDueRank
FROM (
    SELECT
        PurchaseOrderID,
        VendorID,
        OrderDate,
        TaxAmt,
        Freight,
        TotalDue,
        TotalDueRank = ROW_NUMBER () OVER (PARTITION BY VendorID ORDER BY TotalDue DESC)
    FROM Purchasing.PurchaseOrderHeader
) AS Rank
WHERE TotalDueRank <= 3;

(10 rows affected)

Total execution time: 00:00:00.070

PurchaseOrderID,VendorID,OrderDate,TaxAmt,Freight,TotalDue,TotalDueRank
325,1492,2013-04-25 00:00:00.000,119.8008,37.4378,1654.7486,1
1727,1492,2014-01-16 00:00:00.000,61.9164,19.3489,855.2203,2
2517,1492,2014-04-07 00:00:00.000,61.9164,19.3489,855.2203,3
1879,1494,2014-02-04 00:00:00.000,707.784,221.1825,9776.2665,1
1958,1494,2014-02-11 00:00:00.000,707.784,221.1825,9776.2665,2
1800,1494,2014-01-23 00:00:00.000,707.784,221.1825,9776.2665,3
925,1496,2013-09-17 00:00:00.000,165.5413,51.7317,2286.5395,1
1325,1496,2013-12-04 00:00:00.000,165.5413,51.7317,2286.5395,2
397,1496,2013-06-25 00:00:00.000,67.1832,20.9948,927.968,3
2185,1498,2014-03-04 00:00:00.000,2116.422,661.3819,29233.0789,1


In [2]:
USE AW2019;

SELECT TOP 10
    PurchaseOrderID,
    VendorID,
    OrderDate,
    TaxAmt,
    Freight,
    TotalDue,
    TotalDueRank
FROM (
    SELECT
        PurchaseOrderID,
        VendorID,
        OrderDate,
        TaxAmt,
        Freight,
        TotalDue,
        TotalDueRank = DENSE_RANK () OVER (PARTITION BY VendorID ORDER BY TotalDue DESC)
    FROM Purchasing.PurchaseOrderHeader
) AS Rank
WHERE TotalDueRank <= 3;

(10 rows affected)

Total execution time: 00:00:00.037

PurchaseOrderID,VendorID,OrderDate,TaxAmt,Freight,TotalDue,TotalDueRank
325,1492,2013-04-25 00:00:00.000,119.8008,37.4378,1654.7486,1
1727,1492,2014-01-16 00:00:00.000,61.9164,19.3489,855.2203,2
2517,1492,2014-04-07 00:00:00.000,61.9164,19.3489,855.2203,2
3307,1492,2014-06-13 00:00:00.000,61.9164,19.3489,855.2203,2
167,1492,2012-05-30 00:00:00.000,56.8764,17.7739,785.6053,3
1879,1494,2014-02-04 00:00:00.000,707.784,221.1825,9776.2665,1
1958,1494,2014-02-11 00:00:00.000,707.784,221.1825,9776.2665,1
1800,1494,2014-01-23 00:00:00.000,707.784,221.1825,9776.2665,1
1721,1494,2014-01-16 00:00:00.000,707.784,221.1825,9776.2665,1
2116,1494,2014-02-26 00:00:00.000,707.784,221.1825,9776.2665,1


# Scalar Subqueries
- Subqueries that return a single value
- Useful when comparing values in a result set to a scalar value
	- Using aggregate calculations in the WHERE clause for filtering

In [3]:
USE AW2019;

SELECT TOP 10
    BusinessEntityID,
    JobTitle,
    VacationHours,
    MaxVacationHours = (SELECT MAX(VacationHours) FROM HumanResources.Employee),
    PercentOfMaxVacationHours = VacationHours * 1.0 / (SELECT MAX(VacationHours) FROM HumanResources.Employee)
FROM HumanResources.Employee
WHERE VacationHours * 1.0 / (SELECT MAX(VacationHours) FROM HumanResources.Employee) >= 0.8;

(10 rows affected)

Total execution time: 00:00:00.014

BusinessEntityID,JobTitle,VacationHours,MaxVacationHours,PercentOfMaxVacationHours
1,Chief Executive Officer,99,99,1.0000000
27,Production Supervisor - WC60,80,99,0.8080808
40,Production Supervisor - WC60,82,99,0.8282828
48,Production Technician - WC10,83,99,0.8383838
49,Production Technician - WC10,88,99,0.8888888
50,Production Technician - WC10,84,99,0.8484848
51,Production Technician - WC10,85,99,0.8585858
52,Production Technician - WC10,86,99,0.8686868
53,Production Technician - WC10,87,99,0.8787878
54,Production Technician - WC10,89,99,0.8989898


# Correlated Subqueries
- Subqueries that run once for each record in the main/outer query
- Return a scalar output for each of those records
- Can be used in either the SELECT or WHERE clause

In [4]:
USE AW2019;

SELECT TOP 10
    H.PurchaseOrderID,
    H.VendorID,
    H.OrderDate,
    H.TotalDue,
    NonRejectedItems    = (
        SELECT COUNT (*)
        FROM Purchasing.PurchaseOrderDetail AS D
        WHERE D.PurchaseOrderID = H.PurchaseOrderID AND D.RejectedQty = 0
    ),
    MostExpensiveItem   = (
        SELECT MAX (UnitPrice)
        FROM Purchasing.PurchaseOrderDetail AS D
        WHERE D.PurchaseOrderID = H.PurchaseOrderID
    )
FROM Purchasing.PurchaseOrderHeader AS H

(10 rows affected)

Total execution time: 00:00:00.013

PurchaseOrderID,VendorID,OrderDate,TotalDue,NonRejectedItems,MostExpensiveItem
1,1580,2011-04-16 00:00:00.000,222.1492,1,50.26
2,1496,2011-04-16 00:00:00.000,300.6721,2,45.5805
3,1494,2011-04-16 00:00:00.000,9776.2665,1,16.086
4,1650,2011-04-16 00:00:00.000,189.0395,0,57.0255
5,1654,2011-04-30 00:00:00.000,22539.0165,1,37.086
6,1664,2011-04-30 00:00:00.000,16164.0229,1,26.5965
7,1678,2011-04-30 00:00:00.000,64847.5328,3,46.0635
8,1616,2011-04-30 00:00:00.000,766.1827,5,49.644
9,1492,2011-12-14 00:00:00.000,767.0528,5,49.6965
10,1602,2011-12-14 00:00:00.000,1984.6192,3,47.4705


# EXISTS and NOT EXISTS Operators in Correlated Subqueries
- Checks if there is matching record (EXISTS) or not (NOT EXISTS)
- Does not add any fields to our result set, simply returns records where the logic is true
- Similar to joins, but better suited when there is a one-to-many relationship
	- When we need to check for a match from a secondary table and we only want to return a single record from the one side

In [5]:
USE AW2019;

-- EXISTS
SELECT TOP 10
    H.PurchaseOrderID,
    H.OrderDate,
    H.SubTotal,
    H.TaxAmt
FROM Purchasing.PurchaseOrderHeader AS H
WHERE EXISTS (
    SELECT *
    FROM Purchasing.PurchaseOrderDetail AS D
    WHERE D.PurchaseOrderID = H.PurchaseOrderID AND D.OrderQty > 500 AND D.UnitPrice > 50
)
ORDER BY H.PurchaseOrderID;

(10 rows affected)

Total execution time: 00:00:00.032

PurchaseOrderID,OrderDate,SubTotal,TaxAmt
12,2011-12-14 00:00:00.000,34644.225,2771.538
23,2011-12-15 00:00:00.000,37312.275,2984.982
42,2012-01-16 00:00:00.000,34644.225,2771.538
69,2012-01-25 00:00:00.000,91117.95,7289.436
91,2012-02-23 00:00:00.000,34644.225,2771.538
102,2012-03-08 00:00:00.000,35394.975,2831.598
121,2012-03-09 00:00:00.000,34644.225,2771.538
148,2012-04-11 00:00:00.000,91117.95,7289.436
170,2012-05-30 00:00:00.000,34644.225,2771.538
200,2012-06-25 00:00:00.000,34644.225,2771.538


In [6]:
USE AW2019;

-- NOT EXISTS
SELECT TOP 10
    H.PurchaseOrderID,
    H.OrderDate,
    H.SubTotal,
    H.TaxAmt
FROM Purchasing.PurchaseOrderHeader AS H
WHERE NOT EXISTS (
    SELECT *
    FROM Purchasing.PurchaseOrderDetail AS D
    WHERE D.PurchaseOrderID = H.PurchaseOrderID AND D.RejectedQty > 0
)
ORDER BY H.PurchaseOrderID;

(10 rows affected)

Total execution time: 00:00:00.031

PurchaseOrderID,OrderDate,SubTotal,TaxAmt
1,2011-04-16 00:00:00.000,201.04,16.0832
2,2011-04-16 00:00:00.000,272.1015,21.7681
3,2011-04-16 00:00:00.000,8847.30,707.784
5,2011-04-30 00:00:00.000,20397.30,1631.784
6,2011-04-30 00:00:00.000,14628.075,1170.246
7,2011-04-30 00:00:00.000,58685.55,4694.844
8,2011-04-30 00:00:00.000,693.378,55.4702
9,2011-12-14 00:00:00.000,694.1655,55.5332
10,2011-12-14 00:00:00.000,1796.0355,143.6828
11,2011-12-14 00:00:00.000,501.1965,40.0957


# PIVOT Operator
- PIVOT flattens the result set
- Generates a column for each unique value in the pivoted column
- We can apply aggregate functions to the values that fall under the pivoted columns
- *Similar to pivot tables in excel*

In [7]:
USE AW2019;

SELECT
    [Employee Gender] = Gender,
    [Sales Representative], 
    [Buyer], 
    [Janitor]
FROM (
    SELECT 
        JobTitle,
        Gender, 
        VacationHours
    FROM HumanResources.Employee
) AS S
PIVOT (
    AVG (VacationHours)
    FOR JobTitle IN ([Sales Representative], [Buyer], [Janitor])
) AS P
ORDER BY [Employee Gender] ASC;

(2 rows affected)

Total execution time: 00:00:00.015

Employee Gender,Sales Representative,Buyer,Janitor
F,30,54,90
M,31,56,88
